In [ ]:
import torch
from torch import nn
# define customized functions with customized gradients
class STEFunction_relay(torch.autograd.Function):
    """ define straight through estimator with overrided gradient (gate) """
    @staticmethod
    def forward(ctx, input_x, mask, threshold = 0.005):
        ctx.save_for_backward(input_x)
        mask_higher = (input_x > threshold).float()
        mask_lower = (input_x < (-1) * threshold).float()
        mask > 0
        return (input_x > 0).float()

    @staticmethod
    def backward(ctx, grad_output):
        input_x, = ctx.saved_tensors
        return torch.mul(F.softplus(input_x), grad_output)

In [7]:
now_state = 1
in_val = -0.015
threshold = 0.01
def Hysteresis(now_state, in_val, threshold):
    if now_state == 1:
        if in_val < (-1) * threshold:
            now_state = 0
    else:
        if in_val > threshold:
            now_state = 1
    return now_state
now_state = Hysteresis(now_state, in_val, threshold) 
print(now_state)

0


In [40]:
now_state = 1.0
in_val_list = [-0.011, 0.005, 0.001, 0.011, -0.009, -0.011, 0.0, 0.011]
threshold = 0.01
def Hysteresis(now_state, in_val, threshold):
    now_state = now_state * float(in_val > (-1) * threshold) + (1 - now_state) * float(in_val > threshold)
    return now_state
for in_val in in_val_list:
    now_state = Hysteresis(now_state, in_val, threshold) 
    print(now_state)

0.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0


In [22]:
now_state = 0
in_val = -0.0101
threshold = 0.01
now_state = now_state * float(in_val > (-1) * threshold) + (1 - now_state) * float(in_val > threshold)
print(now_state)

0.0


In [43]:
import torch
from torch.nn import functional as F
class MultiInputRelayFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input1, input2):
        ctx.save_for_backward(input1, input2)
        return input1, input2

    @staticmethod
    def backward(ctx, grad_output1, grad_output2):
        input1, input2 = ctx.saved_tensors
        if not input1.requires_grad:
            grad_input1 = None
        else:
            grad_input1 = grad_output1.clone()

        if not input2.requires_grad:
            grad_input2 = None
        else:
            grad_input2 = grad_output2.clone()

        return grad_input1, grad_input2

x1 = torch.randn(10, requires_grad=True)
x2 = torch.randn(10)
print(x1)
# Use the MultiInputRelayFunction to calculate the output
output = MultiInputRelayFunction.apply(x1, x2)

# Calculate the loss
loss = (output[0] ** 2).sum()

# Backprop to calculate gradients
loss.backward()

# Update x1 using an optimizer
optimizer = torch.optim.Adam([x1], lr=0.1)
optimizer.step()
print(x1)

tensor([ 0.3044, -1.3046,  0.2962,  1.8289, -1.2884,  0.0434,  0.2522, -1.1956,
        -0.3554, -0.8371], requires_grad=True)
tensor([ 0.2044, -1.2046,  0.1962,  1.7289, -1.1884, -0.0566,  0.1522, -1.0956,
        -0.2554, -0.7371], requires_grad=True)


In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# define customized functions with customized gradients
class STEFunction_relay(torch.autograd.Function):
    """ define straight through estimator with overrided gradient (gate) """
    @staticmethod
    def forward(ctx, input, mask_old):
        ctx.save_for_backward(input)
        return mask_old

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        return torch.mul(F.softplus(input), grad_output), None

x1 = torch.randn(10, requires_grad=True)
nn.init.uniform_(x1, a = 0, b = 1)
mask_old = (x1 > 0).float()
print(x1)
print(mask_old)
mask_old.requires_grad = False
x1_out = STEFunction_relay.apply(x1, mask_old)
loss = torch.sum(x1_out)
threshold = 0.5

loss.backward()
optimizer = torch.optim.Adam([x1], lr=1)
optimizer.step()
print(x1)
print(mask_old)
mask_new = mask_old * (x1 > (-1) * threshold).float() + (1 - mask_old) * (x1 > threshold).float()
print(mask_new)

tensor([0.7832, 0.8368, 0.3353, 0.2943, 0.4404, 0.9394, 0.7239, 0.0102, 0.3909,
        0.6217], requires_grad=True)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([-0.2168, -0.1632, -0.6647, -0.7057, -0.5596, -0.0606, -0.2761, -0.9898,
        -0.6091, -0.3783], requires_grad=True)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 0., 0., 0., 1., 1., 0., 0., 1.])


In [20]:
import torch

# Create a tensor with random values
tensor = torch.rand(3, 3)

# Define a model with a single linear layer
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(3, 3)

# Initialize the model
model = Model()

# Copy the values from the tensor into the model's parameters
print(model.linear.weight)
print(tensor)
model.linear.weight.data.copy_(tensor)
print(model.linear.weight)

### Other method to copy parameter from one to another
param1 = torch.nn.Parameter(torch.rand(3, 3))
param2 = torch.nn.Parameter(torch.rand(3, 3))
param2.data.copy_(param1.data)
print(param2.data)


Parameter containing:
tensor([[ 0.0808,  0.2326, -0.0325],
        [ 0.4869, -0.0527, -0.3543],
        [ 0.2577, -0.1948, -0.0866]], requires_grad=True)
tensor([[0.8084, 0.4958, 0.9062],
        [0.9594, 0.9866, 0.9635],
        [0.8957, 0.1373, 0.4079]])
Parameter containing:
tensor([[0.8084, 0.4958, 0.9062],
        [0.9594, 0.9866, 0.9635],
        [0.8957, 0.1373, 0.4079]], requires_grad=True)
tensor([[0.8566, 0.1845, 0.7607],
        [0.7572, 0.2837, 0.1152],
        [0.5088, 0.6485, 0.9269]])


In [22]:
class MyClass:
    def __init__(self):
        self._attr1 = Attr1()
    def set_attr1(self, value):
        self._attr1.attr2 = value

class Attr1:
    def __init__(self):
        self.attr2 = None

obj = MyClass()
obj.set_attr1("Value")
print(obj._attr1.attr2)
# setattr(eval('obj._attr1'), 'attr2', 'a')
print(getattr(obj, '_attr1').attr2)
getattr(obj, '_attr1').attr2 = 'a'
print(obj._attr1.attr2)
obj._attr1.attr2 = 'a'
print(obj._attr1.attr2)

Value
Value
a
a
